In [1]:
import pandas as pd
import numpy as np

In [162]:
pd.options.display.float_format = '{:,.2f}'.format

In [154]:
df = pd.read_csv('tether_transactions_522647.csv')
valid = df[df.is_valid == 1]

In [155]:
valid.tx_type.value_counts()

Simple Send               1490224
Crowdsale Purchase             97
Grant Property Tokens          80
Revoke Property Tokens          1
Name: tx_type, dtype: int64

In [166]:
len(valid)

1490402

In [156]:
valid.pivot_table(index='tx_type', values=['amount'], aggfunc=sum)

,amount
tx_type,
Crowdsale Purchase,"113,213.79"
Grant Property Tokens,"2,550,000,000.00"
Revoke Property Tokens,"30,000,000.00"
Simple Send,"56,683,546,721.74"


In [169]:
simple = valid[valid.tx_type == 'Simple Send']
crowdsale = valid[valid.tx_type == 'Crowdsale Purchase']
grant = valid[valid.tx_type == 'Grant Property Tokens']
revoke = valid[valid.tx_type == 'Revoke Property Tokens']

burnt = valid[valid.reference_address.isnull()]

In [170]:
burnt.amount.sum()

30099925.6114425

In [8]:
def get_trans(address, df=valid):
    return df[df.sending_address == address], df[df.reference_address == address]

In [215]:
def compute_summary(df=valid):
    sent = df.pivot_table(index='sending_address',
                          values=['amount', 'block_time', 'is_valid'], 
                          aggfunc={'amount': np.sum, 'block_time': ['first', 'last'], 'is_valid': 'count'})
    sent = sent.reset_index().rename(columns={'sending_address': 'address', 
                                              'amount': 'amount_sent', 
                                              'block_time': 'sent_time', 
                                              'is_valid': 'sent_count'})
    
    recv = df.pivot_table(index='reference_address',
                          values=['amount', 'block_time', 'is_valid'], 
                          aggfunc={'amount': np.sum, 'block_time': ['first', 'last'], 'is_valid': 'count'})
    recv = recv.reset_index().rename(columns={'reference_address': 'address', 
                                              'amount': 'amount_recv', 
                                              'block_time': 'recv_time', 
                                              'is_valid': 'recv_count'})

    out = pd.merge(sent, recv, how='outer', on='address').fillna(0)
    out['balance'] = out['amount_recv'] - out['amount_sent']
    
    out['balance'] = out['balance'].astype(int)
    out['balance'] = out['balance'].astype(float)
    out['sent_count'] = out['sent_count'].astype(int)
    out['recv_count'] = out['recv_count'].astype(int)
    
    return out[['address', 'sent_count', 'recv_count', 'amount_sent', 'amount_recv', 'balance', 'recv_time', 'sent_time']]


In [262]:
def print_detail(address, df=valid):
    sent, recv = get_trans(address, df)
    
    print("Total sent: {:,.2f} ({} transactions to {} unique recepients)".format(sent.amount.sum(), len(sent), sent.reference_address.nunique()))
    print("Total received: {:,.2f} ({} transactionsf from {} unique senders)".format(recv.amount.sum(), len(recv), recv.sending_address.nunique()))
    print("Balance: {:,.2f} tokens".format(recv.amount.sum() - sent.amount.sum()))

In [216]:
%time summary = compute_summary().set_index('address')

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


CPU times: user 56.6 s, sys: 846 ms, total: 57.4 s
Wall time: 58.7 s


In [217]:
summary.sort_values('balance', ascending=False)[:10]

sent_count recv_count      amount_sent  \
                                        count      count              sum   
address                                                                     
1FoWyxwPXuj4C6abqwhjDWdz6D4PZgYRjA     137756     129778 2,817,714,028.02   
1NTMakcgVwQpMdGxRQnFKyb3G1FAJysSfz         35         43 2,075,032,700.00   
1HckjUpRGcrrRAtFaaCAUaGjsPx9oYmLaZ         73      16809 1,168,430,900.00   
1DUb2YYbQA1jjaNYzVXLZ7ZioEhLXtbUru     108846     122235 3,339,888,788.16   
168o1kqNquEJeR9vosUB5fw4eAwcVAgh8P         85      49404 2,516,524,200.00   
1KYiKJEfdJtap9QX2v9BXJMpz2SfU4pgZw      48449      20970 8,228,052,722.36   
1LAnF8h3qMGx3TSwNUHVneBZUEpwE4gu3D     116698       9563 4,124,797,994.03   
1AHeqzQ9VJXAuQu9tTmNqeeo1xgKzFSrTR        218        602   273,839,852.00   
16tg2RJuEPtZooy18Wxn2me2RhUdC94N7r          0          3             0.00   
1MkoWAB4ez4BcZFCStnndUWVM33aotJ1eu          0         50             0.00   

                                        amount_recv        balance  \
                                                sum                  
address                                                              
1FoWyxwPXuj4C6abqwhjDWdz6D4PZgYRjA 3,479,278,626.73 661,564,598.00   
1NTMakcgVwQpMdGxRQnFKyb3G1FAJysSfz 2,385,992,819.30 310,960,119.00   
1HckjUpRGcrrRAtFaaCAUaGjsPx9oYmLaZ 1,395,183,465.53 226,752,565.00   
1DUb2YYbQA1jjaNYzVXLZ7ZioEhLXtbUru 3,557,033,641.77 217,144,853.00   
168o1kqNquEJeR9vosUB5fw4eAwcVAgh8P 2,728,706,598.41 212,182,398.00   
1KYiKJEfdJtap9QX2v9BXJMpz2SfU4pgZw 8,294,644,932.18  66,592,209.00   
1LAnF8h3qMGx3TSwNUHVneBZUEpwE4gu3D 4,180,477,244.33  55,679,250.00   
1AHeqzQ9VJXAuQu9tTmNqeeo1xgKzFSrTR   312,524,609.45  38,684,757.00   
16tg2RJuEPtZooy18Wxn2me2RhUdC94N7r    30,950,000.00  30,950,000.00   
1MkoWAB4ez4BcZFCStnndUWVM33aotJ1eu    29,999,998.90  29,999,998.00   

                                              recv_time                       \
                                                  first                 last   
address                                                                        
1FoWyxwPXuj4C6abqwhjDWdz6D4PZgYRjA  2017-09-19 08:53:34  2018-05-14 15:48:10   
1NTMakcgVwQpMdGxRQnFKyb3G1FAJysSfz  2017-11-26 18:35:33  2018-05-08 14:59:06   
1HckjUpRGcrrRAtFaaCAUaGjsPx9oYmLaZ  2018-01-03 05:39:27  2018-02-22 06:01:27   
1DUb2YYbQA1jjaNYzVXLZ7ZioEhLXtbUru  2015-09-21 19:46:48  2018-05-14 15:48:10   
168o1kqNquEJeR9vosUB5fw4eAwcVAgh8P  2018-02-22 06:03:33  2018-05-14 15:48:10   
1KYiKJEfdJtap9QX2v9BXJMpz2SfU4pgZw  2016-04-27 11:44:49  2018-05-14 15:46:24   
1LAnF8h3qMGx3TSwNUHVneBZUEpwE4gu3D  2017-10-23 11:40:08  2018-05-13 22:41:07   
1AHeqzQ9VJXAuQu9tTmNqeeo1xgKzFSrTR  2018-01-12 10:19:10  2018-05-14 15:48:10   
16tg2RJuEPtZooy18Wxn2me2RhUdC94N7r  2017-11-19 11:10:48  2017-11-19 12:07:03   
1MkoWAB4ez4BcZFCStnndUWVM33aotJ1eu  2018-03-19 01:47:37  2018-05-10 09:02:40   

                                              sent_time                       
                                                  first                 last  
address                                                                       
1FoWyxwPXuj4C6abqwhjDWdz6D4PZgYRjA  2017-09-20 12:31:44  2018-05-14 15:48:10  
1NTMakcgVwQpMdGxRQnFKyb3G1FAJysSfz  2017-11-26 22:01:16  2018-05-12 16:00:04  
1HckjUpRGcrrRAtFaaCAUaGjsPx9oYmLaZ  2018-01-08 07:29:08  2018-02-22 04:51:50  
1DUb2YYbQA1jjaNYzVXLZ7ZioEhLXtbUru  2015-09-21 21:45:20  2018-05-14 15:48:10  
168o1kqNquEJeR9vosUB5fw4eAwcVAgh8P  2018-03-03 04:32:19  2018-05-13 22:41:07  
1KYiKJEfdJtap9QX2v9BXJMpz2SfU4pgZw  2016-04-27 13:02:59  2018-05-14 15:28:39  
1LAnF8h3qMGx3TSwNUHVneBZUEpwE4gu3D  2017-10-23 13:29:38  2018-05-14 15:48:10  
1AHeqzQ9VJXAuQu9tTmNqeeo1xgKzFSrTR  2018-01-15 16:07:24  2018-05-07 15:49:35  
16tg2RJuEPtZooy18Wxn2me2RhUdC94N7r                    0                    0  
1MkoWAB4ez4BcZFCStnndUWVM33aotJ1eu                    0                    0

In [174]:
summary[summary.balance < 0].sort_values('balance')[:10]

,sent_count,recv_count,amount_sent,amount_recv,balance,first_recv,first_sent
address,,,,,,,
3MbYQMMmSkC3AgWkj9FMo5LsPTW1zBTwXL,81,3,"2,580,000,000.00","30,000,015.00","-2,549,999,984.00",2017-10-15 08:57:34,2014-10-06 18:54:05
17ScKNXo4cL8DyfWfcCWu1uJySQuJm7iKC,13136,1,"90,105,062.63",10.00,"-90,105,052.00",2017-10-03 05:50:10,2017-10-03 06:28:04
1DGUWy6N6iHTpussbSP8XrK96tZgDboiWK,4141,13,"53,343,796.25","51,217.72","-53,292,578.00",2017-03-28 08:25:50,2017-03-28 10:07:28
1gJ4FX7n4Udk1LVUSnutAgznyG9JZdQEU,4557,11,"74,918,435.17","31,715,025.90","-43,203,409.00",2017-11-16 15:50:28,2017-11-14 03:53:08
12hnbu1xVuYF4VhaXYFyw6Pq2eudj3CG4g,1439,13,"9,977,391.52","1,531,728.85","-8,445,662.00",2017-12-25 02:29:21,2017-12-25 03:14:42
13coLgY8DpWH7yjunMwyUgjEZQQbrAYT59,301,2,"2,351,433.50","132,750.00","-2,218,683.00",2017-12-22 13:14:47,2017-12-03 22:37:17
1MDVUq9wSSjrrZioy8ajB6Akz8umBnfbGi,6,2,"867,112.00",110.00,"-867,002.00",2017-01-13 00:34:08,2017-01-20 12:54:02
13piDMgYmHJiZmoDELQJcYc5WWDQZB8w9s,50,11,"119,354.57","6,181.30","-113,173.00",2017-03-26 17:06:19,2017-03-26 17:33:30
1Pzz7q2mUqAgbBkTvitPYxuub8WST9iYb3,1,3,"4,412.00",187.10,"-4,224.00",2017-02-26 13:17:09,2018-02-05 22:18:53


In [188]:
grant.reference_address.value_counts()

3BbDtxBSjgfTRxaBUgR2JACWRukLKtZdiQ    38
1NTMakcgVwQpMdGxRQnFKyb3G1FAJysSfz    33
1Nf3oM2pmoKx7M5oNUhyKYr3GLecineMHX     5
132j6EiUWNamSSjiYEQBhsbufsnBh4a28U     2
377UotoWsGk7K2Sx2QCnSVRBhCEZLtQtE4     2
Name: reference_address, dtype: int64

In [182]:
sent, recv = get_trans('31okFF1rUu8jjPEVuajycTRBp82Nteo4Mv')

In [184]:
sent

,tx_hash,block_height,block_hash,block_time,position_in_block,sending_address,reference_address,tx_type,amount,version,is_valid,fee
351893,9e2093f9effac889cca5c232932c8eec9a8d4cf6c3ab8c...,495074,000000000000000000ab08bb37b3cd6abb21707acf2455...,2017-11-19 11:10:48,404,31okFF1rUu8jjPEVuajycTRBp82Nteo4Mv,16tg2RJuEPtZooy18Wxn2me2RhUdC94N7r,Simple Send,"23,000,000.00",0,1,0.00
351953,494e3ed95b4a55c0335a03e9e12f4b3d52af1d247eeb29...,495077,000000000000000000aa01545fd3b6dff0b0af4937f86a...,2017-11-19 11:20:53,170,31okFF1rUu8jjPEVuajycTRBp82Nteo4Mv,16tg2RJuEPtZooy18Wxn2me2RhUdC94N7r,Simple Send,"7,900,000.00",0,1,0.00
352092,3113c473b66133117b5fb7a9161c3b0d9a7248173a4903...,495083,00000000000000000001669c9903c7ee0db739d1f7f58c...,2017-11-19 12:07:03,172,31okFF1rUu8jjPEVuajycTRBp82Nteo4Mv,16tg2RJuEPtZooy18Wxn2me2RhUdC94N7r,Simple Send,"50,000.00",0,1,0.00


In [185]:
recv

,tx_hash,block_height,block_hash,block_time,position_in_block,sending_address,reference_address,tx_type,amount,version,is_valid,fee
351821,6a0774e3a216992326d070f59d0924be30bcf71694c59b...,495073,000000000000000000a62f314542158bfaa173ceda08d5...,2017-11-19 10:53:01,62,3BbDtxBSjgfTRxaBUgR2JACWRukLKtZdiQ,31okFF1rUu8jjPEVuajycTRBp82Nteo4Mv,Simple Send,10.00,0,1,0.00
351822,f68da1d6d78bb1e7bfab503fd91056ad0c5a783dbcb06d...,495073,000000000000000000a62f314542158bfaa173ceda08d5...,2017-11-19 10:53:01,63,3BbDtxBSjgfTRxaBUgR2JACWRukLKtZdiQ,31okFF1rUu8jjPEVuajycTRBp82Nteo4Mv,Simple Send,"1,000,000.00",0,1,0.00
351823,f7b8ae9b2b621919ecb51d7254a9e2b78d9ef64aad266d...,495073,000000000000000000a62f314542158bfaa173ceda08d5...,2017-11-19 10:53:01,64,3BbDtxBSjgfTRxaBUgR2JACWRukLKtZdiQ,31okFF1rUu8jjPEVuajycTRBp82Nteo4Mv,Simple Send,"1,000,000.00",0,1,0.00
351824,40aa4ec86369c0e51e4e5d4c2c9bf503618428acd0cf16...,495073,000000000000000000a62f314542158bfaa173ceda08d5...,2017-11-19 10:53:01,65,3BbDtxBSjgfTRxaBUgR2JACWRukLKtZdiQ,31okFF1rUu8jjPEVuajycTRBp82Nteo4Mv,Simple Send,"1,000,000.00",0,1,0.00
351825,144b272817d55074f9e6c54415e6211296c4702cbd2a64...,495073,000000000000000000a62f314542158bfaa173ceda08d5...,2017-11-19 10:53:01,66,3BbDtxBSjgfTRxaBUgR2JACWRukLKtZdiQ,31okFF1rUu8jjPEVuajycTRBp82Nteo4Mv,Simple Send,"10,000,000.00",0,1,0.00
351826,b168ea72ead7398670bd99b161492202e25bbd6dbeadac...,495073,000000000000000000a62f314542158bfaa173ceda08d5...,2017-11-19 10:53:01,67,3BbDtxBSjgfTRxaBUgR2JACWRukLKtZdiQ,31okFF1rUu8jjPEVuajycTRBp82Nteo4Mv,Simple Send,"10,000,000.00",0,1,0.00
351887,0e34d9a098da348d8c2aa602b232ab00b7d323fd1d25c8...,495074,000000000000000000ab08bb37b3cd6abb21707acf2455...,2017-11-19 11:10:48,72,3BbDtxBSjgfTRxaBUgR2JACWRukLKtZdiQ,31okFF1rUu8jjPEVuajycTRBp82Nteo4Mv,Simple Send,"7,900,000.00",0,1,0.00
352023,1fad99f30bf4e54432380ab2f1be326e2d5d9f14ac63cd...,495081,000000000000000000043cc3b6e2d49338a880cac91dca...,2017-11-19 11:54:04,49,3BbDtxBSjgfTRxaBUgR2JACWRukLKtZdiQ,31okFF1rUu8jjPEVuajycTRBp82Nteo4Mv,Simple Send,"50,000.00",0,1,0.00


In [189]:
print_detail('16tg2RJuEPtZooy18Wxn2me2RhUdC94N7r')

Total sent: 0.00 (0 transactions to 0 unique recepients)
Total received: 30,950,000.00 (3 transactionsf from 1 unique senders)
Balance: 30,950,000.00 tokens


In [204]:
def get_sent(addresses, df=valid, cols=['sending_address', 'reference_address', 'amount']):
    return df[df.sending_address.isin(addresses)][cols]

In [205]:
def get_sent_all(address, df=valid, depth=1):
    sent = get_sent([address], df=df)

    for i in range(depth-1):
        sent = pd.concat([sent, get_sent(list(sent.reference_address), df=df)], axis=0)
        
    return sent

In [253]:
import networkx as nx
import matplotlib.pyplot as plt

from networkx.readwrite import json_graph
from bokeh.util.browser import view

import json, math

In [264]:
def plot_transactions(address, summary, df=valid, depth=1, count_nodes=240):
    g = get_sent_all(address, df, depth) \
            .groupby(['sending_address','reference_address']).agg(np.sum).reset_index()
    g = g.sort_values('amount', ascending=False)[:count_nodes]
    g['amount'] = g['amount'].astype(int)
    g = g.rename(columns={'amount': 'value'})
    
    G = nx.from_pandas_edgelist(g, 'sending_address', 'reference_address', ['value'], create_using=nx.DiGraph())
    pos = nx.kamada_kawai_layout(G)
    
    data = json_graph.node_link_data(G, {'link': 'edges', 'source': 'from', 'target': 'to'})
    
    for node in data['nodes']: 
        size = int(summary.loc[node['id']].balance)
        node['label'] = node['id'][:6] + '...'
        node['title'] = "Balance: {:,d}".format(size)
        node['size'] = 1 if size < 1 else int(math.log(size, 2))
        node['x'] = pos[node['id']][0]
        node['y'] = pos[node['id']][1]
        
        if (node['id'] == address):
            node['color'] = { 'background':'pink', 'border': 'purple' }

    with open('data-notebook.json', 'w') as outfile:
        json.dump(data, outfile)
        
    view('http://0.0.0.0:8000/transactions-notebook.html')

In [265]:
print_detail('3BbDtxBSjgfTRxaBUgR2JACWRukLKtZdiQ')

Total sent: 700,137,097.47 (638 transactions to 284 unique recepients)
Total received: 700,137,124.94 (398 transactionsf from 187 unique senders)
Balance: 27.47 tokens


In [268]:
plot_transactions('1FoWyxwPXuj4C6abqwhjDWdz6D4PZgYRjA', summary, count_nodes=200, depth=4)